# Homework 2 



## Group I

Anna Kurek - 01444623  
Linyun Huang - 01379982  
Mark O'Shea - 01384962  
Mingyang Tham - 01428168  
Rejpal Matharu - 01367169  
Yiting Wang - 01423116

### Question 2 

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


### 1. Load the data into a Python data frame.



In [9]:
file = open('SMSSpamCollection', 'r')
file = file.readlines()
data=pd.DataFrame(file)
data.columns=['x']
data=data['x'].str.split('\t',expand=True)
data.columns=['Label','Message']


 
### 2. Pre-process the SMS messages: Remove all punctuation and numbers from the SMS messages, and change all messages to lower case. (Please provide the Python code that achieves this!)

In [ ]:
data['Message']=data['Message'].str.lower()
data['Message'] = data['Message'].str.replace('[^\w\s]','')
data['Message'] = data['Message'].str.replace('\d+', '')



### 3. Shuffle the messages and split them into a training set (2,500 messages), a validation set (1,000 messages) and a test set (all remaining messages).

In [18]:
test,train, = train_test_split(data, test_size=2500,random_state=12)
test, valid = train_test_split(test, test_size=1000,random_state=123)

### 4. While Python’s SciKit-Learn library has a Naive Bayes classifier, it works with continuous probability distributions and assumes numerical features. 

In [19]:
class NaiveBayesForSpam:
    def train (self, hamMessages, spamMessages):
        self.words = set (' '.join (hamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len (spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len (hamMessages)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) / len (spamMessages)
            self.likelihoods.append ([min (prob1, 0.95), min (prob2, 0.95)])
        self.likelihoods = np.array (self.likelihoods).T
        
    def train2 (self, hamMessages, spamMessages):
        self.words = set (' '.join (hamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len (spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        spamkeywords = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len (hamMessages)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) / len (spamMessages)
            if prob1 * 20 < prob2:
                self.likelihoods.append ([min (prob1, 0.95), min (prob2, 0.95)])
                spamkeywords.append (w)
        self.words = spamkeywords
        self.likelihoods = np.array (self.likelihoods).T

    def predict (self, message):
        posteriors = np.copy (self.priors)
        for i, w in enumerate (self.words):
            if w in message.lower():  # convert to lower-case
                posteriors *= self.likelihoods[:,i]
            else:                                   
                posteriors *= np.ones (2) - self.likelihoods[:,i]
            posteriors = posteriors / np.linalg.norm (posteriors)  # normalise
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]    

    def score (self, messages, labels):
        confusion = np.zeros(4).reshape (2,2)
        for m, l in zip (messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0,0] += 1
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0,1] += 1
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1,0] += 1
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float (confusion.sum()), confusion

### 5. Explain the code: What is the purpose of each function? What do ’train’ and ‘train2’ do, and what is the difference between them? Where in the code is Bayes’ Theorem being applied?

The purpose of the above "train" function is to first calculate the prior probability, and then create a likelihood table for each word (P(word|Spam) & P(word|Ham) in order to further be used to calculate the posterior probability and determine whether they are more likely to be spam or ham using Naive Bayes Algorithm. Notice that each likehood will not exceed 0.95 since the function restricts the highest likelihood to 0.95.

The purpose of the above "train2" function is to first calculate the prior probability of ham and spam message given the number of messages in each category. Next, the function will return a likelihood table just as what "train" function does. However, "train2" function will compare each word's likelihood in spam/ham message and if the Spam prob is 20 times higher the Ham prob, it means that the word has a very high appreance in Spam and thus the function will create a list of these words and classified as Spam key word.

The purpose of the above "predict" function is predict whether a new input message is ham or spam. This function first identify whether each word within the input message is in the spam key word list or not, then further calculate the posterior probability using Naive Bayes Algorithm. This is whether Bayes' Theorem is applied.

The purpose of the above "score" function is to return a confusion matrix specifying the overall predict result from the "predict" function. We can see from the matrix whether the Naive Bayes Classification return the correct classifier of the input message.


### 6. Use your training set to train the classifiers ‘train’ and ‘train2’. Note that the interfaces of our classifiers require you to pass the ham and spam messages separately.



In [20]:
mask = train['Label']=='ham'
ham = list(train[mask]['Message'])
spam = list(train[~mask]['Message'])

m1=NaiveBayesForSpam()

m1.train(ham,spam)

m2=NaiveBayesForSpam()
m2.train2(ham,spam)

### 7. Using the validation set, explore how each of the two classifiers performs out of sample.

In [21]:
valid_messages=list(valid['Message'])
valid_labels=list(valid['Label'])

m1.score(valid_messages, valid_labels)
m2.score(valid_messages, valid_labels)

(0.96499999999999997, array([[ 860.,   31.],
        [   4.,  105.]]))

The accuracy rate of train model is 0.9600.

The accuracy rate of train2 model is 0.9680

Both classifiers have very high accuracy rate, however in this case train classifier performs 
slightly better than train2 classifier.

### 8. Why is the ‘train2’ classifier faster? Why does it yield a better accuracy both on the training and the validation set?

Because the "train2" classifier first filters out the most popular spam word and then conduct the prediction. This makes the algorithm much faster since it does not need to examine each word in spam category. 

### 9. How many false positives (ham messages classied as spam messages) did you get in your validation set? How would you change the code to reduce false positives at the expense of possibly having more false negatives (spam messages classified as ham messages)?

### 10. Run the ‘train2’ classifier on the test set and report its performance using a confusion matrix.


In [25]:
Tmessages=list(test["Message"])
Tlabels=list(test["Label"])

m2.score(Tmessages,Tlabels)

(0.96769527483124396, array([[ 1794.,    56.],
        [   11.,   213.]]))